In [ ]:
from google.genai import types
import base64
from google import genai
import os
import time
import re
from dotenv import load_dotenv

In [ ]:
# Load variables from .env into os.environ
load_dotenv()

In [ ]:
api_key = os.getenv("genai_api_key")
client = genai.Client(
        api_key = api_key,
    )
model = "gemini-1.5-flash-8b" #"gemini-2.5-flash-preview-04-17"

In [ ]:

with open('./myprompt.txt', 'r') as file:
    selected_prompt = file.read()

In [ ]:
image_part1_start = """Original (reference) image:
                    <original_image>"""

image_part1_end = """</original_image>"""

image_part2_start = """Query image (to be identified):
                    <query_image>"""

image_part2_end = """</query_image>"""

generate_content_config = types.GenerateContentConfig(
    response_mime_type="application/json",
    temperature=0.0,
    max_output_tokens=8100,
    top_p=0.95,
    candidate_count=1,
)

In [ ]:
all_main_folder = ['./input_filtered/true',
                   './false/false']
all_file_output = ["./input_filtered/result.txt",
                   './false/result.txt']

In [ ]:
main_folder = all_main_folder[1]  
encoded_images = []

def natural_key(s):
    """Sorts strings in human order, e.g., 1, 2, 10 instead of 1, 10, 2"""
    return [int(text) if text.isdigit() else text.lower() for text in re.split('(\d+)', s)]

for subfolder in sorted(os.listdir(main_folder), key=natural_key):
    subfolder_path = os.path.join(main_folder, subfolder)

    if os.path.isdir(subfolder_path):
        sublist = []

        # Encode all images in the subfolder
        for filename in sorted(os.listdir(subfolder_path), key=natural_key):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                image_path = os.path.join(subfolder_path, filename)
                with open(image_path, "rb") as image_file:
                    encoded = base64.b64encode(image_file.read()).decode('utf-8')
                    sublist.append(encoded)

        if sublist:
            txt_path = os.path.join(subfolder_path, 'original_result.txt')
            sublist.append(txt_path)
            encoded_images.append(sublist)



In [ ]:
file_output = all_file_output[1]
start = 0
end = len(encoded_images)
print(end)
for i in range(start, end):
            image_part1 = types.Part.from_bytes(
                mime_type="image/png", # Assuming JPEG, adjust if needed
                data=base64.b64decode(encoded_images[i][0])
            )
            image_part2 = types.Part.from_bytes(
                mime_type="image/png", # Assuming JPEG, adjust if needed
                data=base64.b64decode(encoded_images[i][1])
            )
            # Create content for the model
            contents = [
                types.Content(
                    role="user",
                    parts=[
                        types.Part.from_text(text=image_part1_start),
                        image_part1, 
                        types.Part.from_text(text=image_part1_end), 
                        types.Part.from_text(text=image_part2_start), 
                        image_part2, 
                        types.Part.from_text(text=image_part2_end),
                        types.Part.from_text(text=selected_prompt)
                    ],
                ),
            ]
            full_response_text = ""
            time.sleep(5)
            for chunk in client.models.generate_content_stream(
                            model=model,
                            contents=contents,
                            config=generate_content_config,
                        ):
                            if chunk.text is not None:
                                full_response_text += chunk.text
            
            
            with open(file_output, 'a', encoding='utf-8') as file:
                    file.write( encoded_images[i][2] + '\n \n' + full_response_text + '\n')
            print(i)
            print(encoded_images[i][2])

In [ ]:
folder_path = "./input_filtered/true/customer111"

# Get all file paths in the folder
image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Ensure you have at least two images
if len(image_files) >= 2:
    encoded_image1 = base64.b64encode(open(image_files[0], "rb").read()).decode('utf-8')
    encoded_image2 = base64.b64encode(open(image_files[1], "rb").read()).decode('utf-8')



# Create parts for the images
image_part1 = types.Part.from_bytes(
    mime_type="image/png", # Assuming JPEG, adjust if needed
    data=base64.b64decode(encoded_image1)
)

image_part2 = types.Part.from_bytes(
    mime_type="image/png", # Assuming JPEG, adjust if needed
    data=base64.b64decode(encoded_image2)
)


# Create content for the model
contents = [
    types.Content(
        role="user",
        parts=[
            types.Part.from_text(text=image_part1_start),
            image_part1, 
            types.Part.from_text(text=image_part1_end), 
            types.Part.from_text(text=image_part2_start), 
            image_part2, 
            types.Part.from_text(text=image_part2_end),
            types.Part.from_text(text=selected_prompt)
        ],
    ),
]


In [ ]:
full_response_text = ""
for chunk in client.models.generate_content_stream(
                model=model,
                contents=contents,
                config=generate_content_config,
            ):
                full_response_text += chunk.text

print(full_response_text)



In [ ]:

import shutil

# Set your main folder path (the one containing subfolders with images)
main_folder = './false'

# Loop through each subfolder
for subfolder in os.listdir(main_folder):
    subfolder_path = os.path.join(main_folder, subfolder)

    if os.path.isdir(subfolder_path):
        for filename in os.listdir(subfolder_path):
            file_path = os.path.join(subfolder_path, filename)

            # Only delete files (not subdirectories)
            if os.path.isfile(file_path):
                os.remove(file_path)
                print(f"Deleted: {file_path}")